## Luke Waninger

In [1]:
from source import *
from IPython.core.interactiveshell import InteractiveShell
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

InteractiveShell.ast_node_interactivity = 'all'

exp   = np.exp
ident = np.identity
na    = np.newaxis
norm  = np.linalg.norm
np.random.seed(42)

cpu_count = multiprocessing.cpu_count()
print(f'cores to utilize:   {cpu_count} of {multiprocessing.cpu_count()}')

cores to utilize:   8 of 8


### The gradient $\triangledown F(\alpha)$ of $F$
for a kernel based support vector machine can be shown to be...

$\triangledown F(\alpha) = 1/n \sum_{i=1}^n \triangledown l(y_i, (K\alpha)_i) + 2\lambda K \alpha$



$
\triangledown l(y,t) =
	\begin{cases}
        0 \hspace{55pt}  yt > 1 + h \\
        -y_i x_i \frac{1+h-yt}{2h} \hspace{18pt}  |1-yt| \le h \\
        -y_i x_i \hspace{40pt}  yt < 1-h
	\end{cases}
$

### Computing the graham and kernel matrices for a Gaussian (RBF) and polynomial kernel
I decided to encapsulate these functions into one class each for radial and polynomial kernels for readability and code reuse. The $\texttt{compute}$ function calculates and returns the requested kernel for any set of observations.

In [2]:
class Kernel(object):
    """tiny base class for SVM kernels"""
    def compute(self):
        yield


class k_radialrbf(Kernel):
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma

    def __str__(self):
        return f'rbf({self.sigma})'

    def compute(self, x, xp=None):
        sigma = self.sigma
        xp = x if xp is None else xp

        def norm(mat):
            return np.linalg.norm(mat, axis=1)

        return exp(-1/(2*sigma**2) * ((norm(x)** 2)[:, na] + (norm(xp)**2)[na, :]-2*(x @ xp.T)))


class k_polynomial(Kernel):
    def __init__(self, degree, b=1.):
        super().__init__()
        self.degree = degree
        self.b = b

    def __str__(self):
        return f'polynomial({self.degree})'

    def compute(self, x, xp=None):
        xp = x if xp is None else xp

        return (x @ xp.T + self.b)**self.degree

### Using scikit learn's builtin digits dataset, download and standardize.

In [3]:
x, y = load_digits(n_class=10, return_X_y=True)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

scalar  = StandardScaler().fit(x_train)
x_train = scalar.transform(x_train)
x_test  = scalar.transform(x_test)

### The support vector machine: $\texttt{mysvm}$
Again, I chose to implement this using OOP so I could encapsulate the SVM, and other helper functions in a single object.

In [4]:
class MySVM(Estimator):
    """kernel based Support Vector Machine"""
    
    def __init__(self, kernel):
        """
        Args:
            kernel (Kernel): to use for transforming the input space

        """
        self.kernel = kernel
    
    def __str__(self):
        return f'SVM(kernel={self.kernel})'
    
    @staticmethod
    def gradient(k, y, beta, l, h=0.5):
        """compute the gradient
        
        Args:
            k (nXn ndarray): computed kernel matrix
            y (1Xn ndarray): true labels
            beta (1Xn ndarray): weight coefficients
            l (float): regularization coefficient
            h (float): [optional, 0 < h < 1] hyperparameter
            
        Returns:
            1Xn ndarray gradient to apply
        """
        n, d = k.shape
        lg = np.zeros([n, d])

        yk = y *(k @ beta)
        mask = np.abs(1 - yk)

        lg[mask <= h] = ((1/(2*h)) * ((1 + h-yk)[:, na]) * (-y[:, na] * k))[mask <= h]
        lg[yk < 1-h]  = (-y[:, na] * k)[yk < 1-h]

        return np.array(np.sum(lg, axis=0)/n + 2*l*beta)
    
    def fgrad(self, k, y, l, eta=1., max_iter=100):
        """fast gradient descent
        
        Args:
            k (nXn ndarray): computer kernel matrix
            y (1Xn ndarray): true labels
            l (float): regularization coefficient
            eta (float): [optional, 0 < eta < 1] learning rate
            max_iter (int): [optional, max_iter > 1]: maximum learning iterations
            
        Returns:
            1Xn ndarray of optimized weight coefficients
        """
        n, d  = k.shape
        b0    = np.zeros(d)
        theta = np.copy(b0)
        grad  = self.gradient(k, y, b0, l)
        i = 0
        
        # continue optimizing until either the max iterations is reached
        # or gradient stops moving
        while i < max_iter and not np.isclose(0, eta):
            eta = backtracking(k, y, b0, l, eta, self.gradient, self.objective)

            b1 = theta - eta*grad
            theta = b1 + (i/(i+3))*(b1-b0)
            grad  = self.gradient(k, y, theta, l)
            b0 = b1

            i += 1

        return b0
    
    @staticmethod
    def objective(k, y, l, beta, h=0.5):
        """objective function
        
        Args:
            k (nXn ndarray): computed kernel matrix
            y (1Xn ndarray): true labels
            beta (1Xn ndarray): weight coefficients
            l (float): regularization coefficient
            h (float): [optional, 0 < h < 1] hyperparameter
            
        Returns:
            float, loss
        """
        n, d = k.shape
        loss = np.zeros(n)
        yk = y * (k @ beta)
        mask = np.abs(1 - yk)

        loss[mask <= h] = ((1 + h-yk)**2 / (4*h))[mask <= h]
        loss[yk < 1-h] = (1 - yk)[yk < 1-h]

        return np.sum(loss)/n + l*norm(beta)**2

    def predict(self, kp, beta):
        """predict labels
        
        Args:
            kp (nXn ndarray): computed kernel matrix
            beta (1Xn ndarray): weight coefficients
        
        Returns:
            1Xn ndarray of predicted labels
        """
        return [1 if ki @ beta.T > 0 else -1 for ki in kp]
    
    def predict_proba(self, kp, beta):
        """predict probabilities
        
        Args:
            kp (nXn ndarray): computed kernel matrix
            beta (1Xn ndarray): weight coefficients
        
        Returns:
            1Xn ndarray of predicted probabilities
        """
        return [ki @ beta.T for ki in kp]

### Training the SVM with the huberized hinge loss and an order 7 polynomial kernel

running one vs rest with a polynomial kernel of degree 7 with $\lambda$=1 gives a horrible validation error: $\approx$ 0.518

In [5]:
kernl = k_polynomial(7)
OVR(MySVM(kernl), n_jobs=-1).fit(x_train, y_train, x_test, y_test, 1.)

fitting <OVR(estimator=SVM(kernel=polynomial(7)) err=0.0)>:  90%|█████████ | 9/10 [01:33<00:13, 14.00s/it]

<OVR(estimator=SVM(kernel=polynomial(7)) err=0.5183501683501682)>

Using cross-validation we see a performance improvement of around 9%!

In [6]:
ovr = OVR(MySVM(kernl), n_jobs=-1)
cv(x_train, y_train, ovr, eargs=np.linspace(.001, 1., 5), nfolds=3)

3-Fold CV: <OVR(estimator=SVM(kernel=polynomial(7)) err=0.0)>:  98%|█████████▊| 49/50 [00:47<00:00,  1.05it/s]

<OVR(estimator=SVM(kernel=polynomial(7)) err=0.42966506201657173)>

### Comparing the performance of kernel SVMs
It quickly becomes clear the 7-degree polynomial kernel is a horrible choice. Below, I run a series of OVO polynomial and radial kernels that all show much better performance. 

In [ ]:
kernels = [
    k_polynomial(1),
    k_radialrbf(.1),
    k_radialrbf(1),
]

cvs = []
for kernl in kernels:
    ovr, eargs = OVR(MySVM(kernl), n_jobs=-1), np.linspace(.001, 1., 3)
    cvs.append(cv(x_train, y_train, ovr, eargs, nfolds=3))

cvs

3-Fold CV: <OVR(estimator=SVM(kernel=rbf(1)) err=0.0)>:  40%|████      | 20/50 [00:22<00:20,  1.49it/s]s]s/it]